In [54]:
import multiprocessing
import os
import tempfile
import uuid
import warnings

import torch
from torch import nn
from torchrl.collectors import MultiaSyncDataCollector
from torchrl.data import LazyMemmapStorage, MultiStep,TensorDictReplayBuffer
from torchrl.envs import (
    EnvCreator,
    ExplorationType,
    ParallelEnv,
    RewardScaling,
    StepCounter,
)
from torchrl.envs.libs.gym import GymEnv
from torchrl.envs.transforms import (
    CatFrames,
    Compose,
    GrayScale,
    ObservationNorm,
    Resize,
    ToTensorImage,
    TransformedEnv,
)
from torchrl.modules import DuelingCnnDQNet, EGreedyModule, QValueActor
from tensordict.nn import TensorDictSequential


from torchrl.objectives import DQNLoss, SoftUpdate
from torchrl.record.loggers.csv import CSVLogger
from torchrl.trainers import (
    LogReward,
    Recorder,
    ReplayBufferTrainer,
    Trainer,
    UpdateWeights
)

In [55]:
is_fork = multiprocessing.get_start_method() == "fork"
device = torch.device("cpu")
# the learning rate of the optimizer
lr = 2e-3
# weight decay
wd = 1e-5
# the beta parameters of Adam
betas = (0.9, 0.999)
# Optimization steps per batch collected (aka UPD or updates per data)
n_optim = 8


# gamma decay factor
gamma = 0.99

# Smooth target network update decay parameter.
# This loosely corresponds to a 1/tau interval with hard target network update
tau = 0.02

###############################################################################
# Data collection and replay buffer
# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
#
# .. note::
#   Values to be used for proper training have been commented.
#
# Total frames collected in the environment. In other implementations, the
# user defines a maximum number of episodes.
# This is harder to do with our data collectors since they return batches
# of N collected frames, where N is a constant.
# However, one can easily get the same restriction on the number of episodes by
# breaking the training loop when a certain number of episodes has been collected.
total_frames = 5_000  # 500000

# Random frames used to initialize the replay buffer.
init_random_frames = 100  # 1000

# Frames in each batch collected.
frames_per_batch = 32  # 128

# Frames sampled from the replay buffer at each optimization step
batch_size = 32  # 256

# Size of the replay buffer in terms of frames
buffer_size = min(total_frames, 100000)

# Number of environments run in parallel in each data collector
num_workers = 2  # 8
num_collectors = 2  # 4

###############################################################################
# Environment and exploration
# ~~~~~~~~~~~~~~~~~~~~~~~~~~~
#
# We set the initial and final value of the epsilon factor in Epsilon-greedy exploration.
# Since our policy is deterministic, exploration is crucial: without it, the
# only source of randomness would be the environment reset.

eps_greedy_val = 0.1
eps_greedy_val_env = 0.005

# To speed up learning, we set the bias of the last layer of our value network
# to a predefined value (this is not mandatory)
init_bias = 2.0


In [56]:
def make_env(
    parallel=False,
    obs_norm_sd=None,
):
    if obs_norm_sd is None:
        obs_norm_sd = {"standard_normal": True}
    if parallel:
        base_env = ParallelEnv(
            num_workers,
            EnvCreator(
                lambda: GymEnv(
                    "CartPole-v1",
                    from_pixels=True,
                    pixels_only=True,
                    device=device,
                )
            ),
        )
    else:
        base_env = GymEnv(
            "CartPole-v1",
            from_pixels=True,
            pixels_only=True,
            device=device,
        )

    env = TransformedEnv(
        base_env,
        Compose(
            StepCounter(),  # to count the steps of each trajectory
            ToTensorImage(),
            RewardScaling(loc=0.0, scale=0.1),
            GrayScale(),
            Resize(64, 64),
            CatFrames(4, in_keys=["pixels"], dim=-3),
            ObservationNorm(in_keys=["pixels"], **obs_norm_sd),
        ),
    )
    return env

In [57]:
def get_norm_stats():
    test_env = make_env()
    test_env.transform[-1].init_stats(
        num_iter=1000, cat_dim=0, reduce_dim=[-1, -2, -4], keep_dims=(-1, -2)
    )
    obs_norm_sd = test_env.transform[-1].state_dict()
    # let's check that normalizing constants have a size of ``[C, 1, 1]`` where
    # ``C=4`` (because of :class:`~torchrl.envs.CatFrames`).
    print("state dict of the observation norm:", obs_norm_sd)
    test_env.close()
    return obs_norm_sd
# get_norm_stats()

In [58]:
def make_model(dummy_env):
    cnn_kwargs = {
        "num_cells": [32, 64, 64],
        "kernel_sizes": [6, 4, 3],
        "strides": [2, 2, 1],
        "activation_class": nn.ELU
    }
    mlp_kwargs = {
        "depth": 2,
        "num_cells": [64, 64],
        "activation_class": nn.ELU,
    }
    net = DuelingCnnDQNet(dummy_env.action_spec.shape[-1], 1, cnn_kwargs, mlp_kwargs).to(device)
    net.value[-1].bias.data.fill_(init_bias)
    actor = QValueActor(net, in_keys=["pixels"], spec=dummy_env.action_spec).to(device)
    # init LazyNets
    tensordict = dummy_env.fake_tensordict()
    actor(tensordict)
    
    # we joint our actor with an EGreedyModule for data collection
    exploration_module = EGreedyModule(
        spec=dummy_env.action_spec,
        annealing_num_steps=total_frames,
        eps_init=eps_greedy_val,
        eps_end=eps_greedy_val_env
    )
    actor_explore = TensorDictSequential(actor, exploration_module)
    return actor, actor_explore

In [59]:
def get_replay_buffer(buffer_size, n_optim, batch_size):
    replay_buffer = TensorDictReplayBuffer(
        batch_size=batch_size,
        storage=LazyMemmapStorage(buffer_size),
        prefetch=n_optim,
    )
    return replay_buffer

In [60]:
def get_collector(
    stats,
    num_collectors,
    actor_explore,
    frames_per_batch,
    total_frames,
    device,
):
    cls = MultiaSyncDataCollector
    env_arg = [make_env(parallel=True, obs_norm_sd=stats)] * num_collectors
    data_collector = cls(
        env_arg,
        policy=actor_explore,
        frames_per_batch=frames_per_batch,
        total_frames=total_frames,
        # this is the default behaviour: the collector runs in ``"random"`` (or explorative) mode
        exploration_type=ExplorationType.RANDOM,
        # We set the all the devices to be identical. Below is an example of
        # heterogeneous devices
        device=device,
        storing_device=device,
        split_trajs=False,
        postproc=MultiStep(gamma=gamma, n_steps=5),
    )
    return data_collector

In [61]:
def get_loss_module(actor, gamma):
    loss_module = DQNLoss(actor, delay_value=True)
    loss_module.make_value_estimator(gamma=gamma)
    target_updater = SoftUpdate(loss_module, eps=0.995)
    return loss_module, target_updater

In [62]:
stats = get_norm_stats()
# how come here the parallel is False and above the parallel be true?
test_env = make_env(parallel=False, obs_norm_sd=stats)
# Get model
actor, actor_explore = make_model(test_env)
loss_module, target_net_updater = get_loss_module(actor, gamma)

collector = get_collector(
    stats=stats,
    num_collectors=num_collectors,
    actor_explore=actor_explore,
    frames_per_batch=frames_per_batch,
    total_frames=total_frames,
    device=device,
)
optimizer = torch.optim.Adam(
    loss_module.parameters(), lr=lr, weight_decay=wd, betas=betas
)
exp_name = f"dqn_exp_{uuid.uuid1()}"
tmpdir = tempfile.TemporaryDirectory()
logger = CSVLogger(exp_name=exp_name, log_dir=tmpdir.name)
warnings.warn(f"log dir: {logger.experiment.log_dir}")

/home/kostiskak/anaconda3/envs/rl_env/lib/python3.11/site-packages/gymnasium/core.py:311: UserWarning: WARN: env.num_envs to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.num_envs` for environment variables or `env.get_wrapper_attr('num_envs')` that will search the reminding wrappers.
  logger.warn(
/home/kostiskak/anaconda3/envs/rl_env/lib/python3.11/site-packages/gymnasium/core.py:311: UserWarning: WARN: env.reward_space to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.reward_space` for environment variables or `env.get_wrapper_attr('reward_space')` that will search the reminding wrappers.
  logger.warn(


state dict of the observation norm: OrderedDict([('standard_normal', tensor(True)), ('loc', tensor([[[0.9895]],

        [[0.9895]],

        [[0.9895]],

        [[0.9895]]])), ('scale', tensor([[[0.0737]],

        [[0.0737]],

        [[0.0737]],

        [[0.0737]]]))])


/home/kostiskak/anaconda3/envs/rl_env/lib/python3.11/site-packages/gymnasium/core.py:311: UserWarning: WARN: env.num_envs to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.num_envs` for environment variables or `env.get_wrapper_attr('num_envs')` that will search the reminding wrappers.
  logger.warn(
/home/kostiskak/anaconda3/envs/rl_env/lib/python3.11/site-packages/gymnasium/core.py:311: UserWarning: WARN: env.reward_space to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.reward_space` for environment variables or `env.get_wrapper_attr('reward_space')` that will search the reminding wrappers.
  logger.warn(
/home/kostiskak/anaconda3/envs/rl_env/lib/python3.11/site-packages/torch/nn/modules/lazy.py:181: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.


In [63]:
log_interval = 500

trainer = Trainer(
    collector=collector,
    total_frames=total_frames,
    frame_skip=1,
    loss_module=loss_module,
    optimizer=optimizer,
    logger=logger,
    optim_steps_per_batch=n_optim,
    log_interval=log_interval
)

buffer_hook = ReplayBufferTrainer(
    get_replay_buffer(buffer_size, n_optim, batch_size=batch_size),
    flatten_tensordicts=True,
)
buffer_hook.register(trainer)
weight_updater = UpdateWeights(collector, update_weights_interval=1)
weight_updater.register(trainer)
recorder = Recorder(
    record_interval=100,  # log every 100 optimization steps
    record_frames=1000,  # maximum number of frames in the record
    frame_skip=1,
    policy_exploration=actor_explore,
    environment=test_env,
    exploration_type=ExplorationType.MODE,
    log_keys=[("next", "reward")],
    out_keys={("next", "reward"): "rewards"},
    log_pbar=True,
)
recorder.register(trainer)

trainer.register_op("post_steps", actor_explore[1].step, frames=frames_per_batch)
trainer.register_op("post_optim", target_net_updater.step)

log_reward = LogReward(log_pbar=True)
log_reward.register(trainer)

In [64]:
trainer.train()

  0%|          | 0/5000 [00:00<?, ?it/s]/home/kostiskak/ntua/Robotalk/rl/torchrl/envs/utils.py:160: UserWarning: The expected key set and actual key set differ. This will work but with a slower throughput than when the specs match exactly the actual key set in the data. Expected - Actual keys=set(), 
Actual - Expected keys={'chosen_action_value', 'action_value'}.
  warnings.warn(
r_training: 0.4948, rewards: 0.1000, total_rewards: 5.2632: : 5024it [01:27, 68.60it/s]                        